<font  style="font-size: 3rem; color: darkviolet"> Convolutional Neural Networks in TensorFlow - *part 1* </font>

DEL - 2023/24 - TP2 (1h30)

*This assignement is inspired by the Deep Learning course on Coursera by Andrew Ng, Stanford University, for which we are thankful.*

In this assignment, you will gain practical experience in constructing and training Convolutional Neural Networks (ConvNets) using the TensorFlow Keras Sequential API. This API offers an intuitive and straightforward approach to constructing and training ConvNets within the TensorFlow framework. It is suited for tasks that involve a sequential flow, where each layer has precisely one input tensor and one output tensor.

You can access the documentation for the Sequential API at the following link: https://www.tensorflow.org/guide/keras/sequential_model

You will develop a binary classifier utilizing the Sequential API to determine the emotional state of an individual as either positive or negative. 

### Table of Contents
- [1 - The Happy House Dataset](#1)
- [2 - Creating the Sequential Model](#2)
- [3 - Training and Evaluating the Model](#3)

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.layers as tfl

from data.test_utils import summary, comparator

%matplotlib inline
np.random.seed(1)

<a name='1'></a>
## <font color='darkviolet'> 1 - The Happy House Dataset

The Happy House dataset is a collection of facial images, and your task is to develop a ConvNet capable of effectively determining whether the individuals in these images are smiling or not. This task holds significance, as only those with a genuine smile will gain access to the Happy House.

In [2]:
# Run the following lines to load the training and test datasets
train_dataset = h5py.File('data/train_happy.h5', "r")
test_dataset = h5py.File('data/test_happy.h5', "r")

# Retrieve the keys within a HDF5 file
dataset_keys = list(train_dataset.keys())
print(dataset_keys[:])

['list_classes', 'train_set_x', 'train_set_y']


<a name='ex-1'></a>
### <font color='blue'> Exercise 1 - load_happy_dataset

<font color='blue'>**1.1** <font color='black'> Implement a function, `load_happy_dataset`, that loads both the training and test datasets from the provided external files: extract the features and labels for both sets and return them as NumPy arrays. Ensure that you reshape the labels to match the expected shape (m, 1), where 'm' is the number of examples.

In [3]:
def load_happy_dataset():
    train_dataset = h5py.File('data/train_happy.h5', "r")
    test_dataset = h5py.File('data/test_happy.h5', "r")
    train_features = train_dataset  
    return 
    #TODO

IndentationError: expected an indented block (3289724065.py, line 3)

<font color='blue'>**1.2** <font color='black'> Normalise the input images by scaling pixel values to the range [0, 1].

In [ ]:
#TODO

<font color='blue'>**1.3** <font color='black'> Provide a description of your dataset, including its size, dimensions, labels, and the distribution of labels. Display a few images to visualize the dataset.

In [ ]:
#TODO

<a name='2'></a>
## <font color='darkviolet'> 2 - Creating the Sequential Model

Constructing a Sequential model in Keras entails assembling a sequence of layers within the Sequential constructor. These layers collectively define the architecture of your ConvNet, and the order in which you arrange them determines the sequence of transformations applied to the input data.

It's important to note that in Keras, you must specify the input shape for your model. This is because the shape of the layer weights is determined by the shape of the inputs.

<a name='ex-2'></a>
### <font color='blue'> Exercise 2 - happyModel

<font color='blue'>**2.1** <font color='black'>Implement the `happyModel` function to create a specific ConvNet model with the following layers: `ZeroPadding2D -> Conv2D -> BatchNormalization -> ReLU -> MaxPooling2D -> Flatten -> Dense`.
    
You can use the documentation for reference: [tf.keras.layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers).

Here are the configuration details for each layer:
    
 - ZeroPadding2D: apply padding of 3 pixels to the input shape.
 - Conv2D: use 32 filters of size 7x7 with a stride of 1.
 - BatchNormalization: perform batch normalization along the depth axis (i.e., the channels or feature maps).
 - ReLU: apply Rectified Linear Unit activation function.
 - MaxPool2D: use default parameters for max pooling.
 - Flatten: flatten the previous output.
 - Fully-connected (Dense) layer: add a fully connected layer with 1 neuron and a sigmoid activation.

You can introduce these layers into a Sequential model using the `.add()` method. 
    
Note: Batch Normalization (BN) normalizes the inputs to a layer by subtracting the mean and dividing by the standard deviation of the mini-batch. This centers the data around zero and scales it to have a standard deviation of one. The mean and variance for each channel are computed during training and remain constant during inference. After normalization, BN introduces learnable parameters, gamma and beta, for each channel in the layer. Gamma values allow the network to increase or decrease the importance of each channel's features, while the beta parameter enables fine adjustments to the normalized values for a better fit to the training data.

In [ ]:
def happyModel():
    
    #TODO

In [ ]:
# Test the model implementation
happy_model = happyModel()
# Print a summary for each layer
for layer in summary(happy_model):
    print(layer)
# The expected layer configurations
output = [['ZeroPadding2D', (None, 70, 70, 3), 0, ((3, 3), (3, 3))],
            ['Conv2D', (None, 64, 64, 32), 4736, 'valid', 'linear', 'GlorotUniform'],
            ['BatchNormalization', (None, 64, 64, 32), 128],
            ['ReLU', (None, 64, 64, 32), 0],
            ['MaxPooling2D', (None, 32, 32, 32), 0, (2, 2), (2, 2), 'valid'],
            ['Flatten', (None, 32768), 0],
            ['Dense', (None, 1), 32769, 'sigmoid']]
comparator(summary(happy_model), output)

<font color='blue'>**2.2** <font color='black'> After designing the deep learning model, the next step is to compile it for training. In the context of TensorFlow and other deep learning frameworks, compiling a model involves configuring various settings that dictate how the training process will proceed. Set the following configurations:

- Optimizer: Select the "Adam" optimizer with a learning rate of 0.0001.
- Loss Function: Specify the "binary_crossentropy" loss function, ideal for binary classification tasks like distinguishing between smiling and non-smiling faces.
- Metrics: Monitor the "accuracy" metric during the training process to assess model performance.

In [ ]:
happy_model.compile(#TODO)

<font color='blue'> **Q2.1** Execute the following line: `happy_model.summary()`. What is the distinction between trainable and non-trainable parameters? Can you identify the source of the 64 non-trainable parameters?
    
<font color='blue'> **Q2.2** How are the total parameters reported per layer calculated? Make the computations and verify your results with the `happy_model.summary()` ouput. 
    
<font color='blue'> **Q2.3** You might have noticed that the first output dimension of a layer is labeled as "None." Can you provide an explanation for this?

<a name='3'></a>
## <font color='darkviolet'> 3 - Training and Evaluating the Model

<font color='blue'>**3.1** <font color='black'> After creating and compiling your model, the next step is to initiate the training process. This is accomplished by calling `.fit()`. This method leverages the underlying TensorFlow framework to automate various aspects of the training process, including backpropagation.

The training process may include 20 epochs, a batch size of 32, and a validation split of 80% for training and 20% for validation to monitor the model's performance.

You can use the **`ModelCheckpoint`** callback in conjunction with `.fit()` to specify criteria for saving the best model automatically. You can find more details about this callback in the Keras documentation https://keras.io/api/callbacks/model_checkpoint/. Additionally, you can use **`EarlyStopping`** in combination with `ModelCheckpoint`. It helps prevent overfitting by monitoring a specific metric, such as validation accuracy, and stopping training when this metric no longer improves for a certain number of epoch. More information about this callback can be found in the Keras documentation https://keras.io/api/callbacks/early_stopping/.
    
Note: The .fit() method in Keras returns a History object. This object contains information about the training process.

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

#TODO

<font color='blue'> **Q3.1** Observe the training history, including the changes in loss and accuracy over each epoch, for the training and validation dataset. Describe.

<font color='blue'>**3.2** <font color='black'>After training your model using `.fit()`, you can evaluate its performance on your test set by calling `.evaluate()`. This function returns the value of the loss function and any performance metrics you specified during compilation. The evaluation provides insights into **how well your model generalizes to unseen data and helps you assess its overall performance**. 

In [ ]:
#TODO

<font color='blue'> **Q3.2** How does the performance on the test dataset compare to the training performance you observed during the training process?